In [19]:
def loadData():
    rating_data={1: {'A': 5, 'B': 3, 'C': 4, 'D': 4},
                 2: {'A': 3, 'B': 1, 'C': 2, 'D': 3, 'E': 3},
                 3: {'A': 4, 'B': 3, 'C': 4, 'D': 3, 'E': 5},
                 4: {'A': 3, 'B': 3, 'C': 1, 'D': 5, 'E': 4},
                 5: {'A': 1, 'B': 5, 'C': 5, 'D': 2, 'E': 1}
              }
    return rating_data	
users_rating = loadData()
users_rating

{1: {'A': 5, 'B': 3, 'C': 4, 'D': 4},
 2: {'A': 3, 'B': 1, 'C': 2, 'D': 3, 'E': 3},
 3: {'A': 4, 'B': 3, 'C': 4, 'D': 3, 'E': 5},
 4: {'A': 3, 'B': 3, 'C': 1, 'D': 5, 'E': 4},
 5: {'A': 1, 'B': 5, 'C': 5, 'D': 2, 'E': 1}}

In [5]:
# 建立倒排索引
items_rating = {}
for user,ratings in users_rating.items():
    for item,rating in ratings.items():
        if item not in items_rating:
            items_rating[item] = {}
        if user not in items_rating[item]:
            items_rating[item][user] = 0
        items_rating[item][user] = rating
items_rating

{'A': {1: 5, 2: 3, 3: 4, 4: 3, 5: 1},
 'B': {1: 3, 2: 1, 3: 3, 4: 3, 5: 5},
 'C': {1: 4, 2: 2, 3: 4, 4: 1, 5: 5},
 'D': {1: 4, 2: 3, 3: 3, 4: 5, 5: 2},
 'E': {2: 3, 3: 5, 4: 4, 5: 1}}

In [11]:
# 计算物品间评分偏差
Ratings_diffs = {}
N_set = {}
for itemx,itemx_history in items_rating.items():
    if itemx not in Ratings_diffs:
        Ratings_diffs[itemx] = {}
        N_set[itemx] = {}
    for itemy,itemy_history in items_rating.items():
        if itemx != itemy:
            Ratings_diffs[itemx][itemy] = 0
            N_set[itemx][itemy] = 0
            for x_user in itemx_history:
                if x_user in itemy_history:
                    Ratings_diffs[itemx][itemy] += items_rating[itemy][x_user] - items_rating[itemx][x_user]
                    N_set[itemx][itemy] += 1
for itemx,ys in Ratings_diffs.items():
    for itemy,rating in ys.items():
        Ratings_diffs[itemx][itemy] /= N_set[itemx][itemy]

Ratings_diffs,N_set

({'A': {'B': -0.2, 'C': 0.0, 'D': 0.2, 'E': 0.5},
  'B': {'A': 0.2, 'C': 0.2, 'D': 0.4, 'E': 0.25},
  'C': {'A': 0.0, 'B': -0.2, 'D': 0.2, 'E': 0.25},
  'D': {'A': -0.2, 'B': -0.4, 'C': -0.2, 'E': 0.0},
  'E': {'A': -0.5, 'B': -0.25, 'C': -0.25, 'D': 0.0}},
 {'A': {'B': 5, 'C': 5, 'D': 5, 'E': 4},
  'B': {'A': 5, 'C': 5, 'D': 5, 'E': 4},
  'C': {'A': 5, 'B': 5, 'D': 5, 'E': 4},
  'D': {'A': 5, 'B': 5, 'C': 5, 'E': 4},
  'E': {'A': 4, 'B': 4, 'C': 4, 'D': 4}})

In [17]:
# 预测评分
# 首先找出Alice交互过的物品哪些与要预测的物品有过”共同被统一用户评分“的经历，即存在倒排索引Ratings_item[x][y]
A_history = users_rating[1]
candidate_items = set()
for item in Ratings_diffs['E']:
    if item in A_history:
        candidate_items.add(item)
weighted_score = 0
weighted_sum = 0
for item in Ratings_diffs['E']:
    weighted_sum += N_set['E'][item]
    weighted_score += Ratings_diffs['E'][item] * N_set['E'][item]
predict = weighted_score/weighted_sum
predict

-0.25